In [3]:
from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader
import gradio as gr
import os
import json

/home/hp/AI-Agents/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
load_dotenv()
openai = OpenAI()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [5]:
reader = PdfReader("Ahmed-Saber-CV.pdf")
data = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        data += text

reader = PdfReader("GitHub skills extraction.pdf")
github_skills = "github skills:\n"
for page in reader.pages:
    text = page.extract_text()
    if text:
        github_skills += text

In [10]:
name = "Ahmed Saber"

In [11]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and github repositories content and projects which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{data}\n\n## github:\n{github_skills}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [12]:
def chat(message , history):
    messages = [{"role": "system" , "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat , type="messages").launch()

In [13]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable : bool
    feedback: str

In [ ]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Data:\n{data}\n\n## Github:\n{github_skills}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."
evaluator_system_prompt += "With this context, make sure that the response always sound humble and simple and professional using adjective that represent me as a hard worker person and love to discover new things dont let the response has a sound like and expert did you understand ?"
# Strict JSON response requirement for machine parsing
_evaluator_json_instruction = (
    "Respond ONLY with a valid JSON object on a single line with keys "
    "'is_acceptable' (boolean) and 'feedback' (string). No markdown, no code fences, no extra text."
)
evaluator_system_prompt += "\n\n" + _evaluator_json_instruction

print(evaluator_system_prompt)

In [23]:
def evaluate_user_prompt(reply , message , history):
    user_prompt = f"Here is the conversation between the User and the Agent: \n\n{history}"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [37]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [38]:
ollama = OpenAI(base_url="http://localhost:11434/v1", api_key='ollama')
model_name = "llama3.2"


In [16]:
def evaluate(reply , message , history) -> Evaluation:
    messages = [
        {"role": "system", "content": evaluator_system_prompt},
        {"role": "user", "content": evaluate_user_prompt(reply, message, history)},
    ]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    text = response.choices[0].message.content
    try:
        data = json.loads(text)
        return Evaluation.model_validate(data)
    except Exception:
        # Fallback: treat raw text as feedback and mark as not acceptable
        return Evaluation(is_acceptable=False, feedback=text)

In [17]:
def rerun(reply, message , history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
def chat(message, history):
    system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply = response.choices[0].message.content

    evaluation = evaluate(|reply, message, history)
    print(evaluation)
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


is_acceptable=True feedback='The response is well-structured and effectively communicates the technical skills of Ahmed Saber. It maintains a professional tone while providing detailed information about his expertise. However, simplifying the language slightly and emphasizing humility, such as mentioning a willingness to learn and collaborate, would enhance the engagement without coming across as overly expert. Overall, it is a solid and acceptable response.'
Passed evaluation - returning reply
is_acceptable=False feedback='The response is informative and highlights achievements, but it could come across as overly confident. It should reflect more humility and a focus on a love for learning and exploration rather than expertise. Consider simplifying the language and expressing how these experiences are part of a continuous journey of growth and discovery.'
Failed evaluation - retrying
The response is informative and highlights achievements, but it could come across as overly confident.